### Important
The taboo_data folder is not included, but is needed for creating the human and original data csv files. For where to download, please check the root README.md

In [1]:
import os
import openai

import pandas as pd
import os
import numpy as np
from ast import literal_eval
import json
import pickle
import time

In [2]:
default_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}"'

In [3]:
openai.api_key = # removed, paste your own here

In [7]:
def request_response_from_gpt(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        #prompt=prompt,
        messages=[
            {"role": "system", "content": "You are a crowdsourcing worker that earns a living through creating paraphrases."},
            {"role": "user", "content": prompt}],
        temperature=1,
        frequency_penalty=0.0,
        presence_penalty=1.5,
        n=1)
    return response

In [8]:
def request_with_checks(prompt):
    success = False
    count = 0
    while not success:
        if count > 0:
            print(f'Retrying with again. Current number of retries: {count}')
        if count >= 5:
            raise Exception('Too many attempts')
        try:
            response = request_response_from_gpt(prompt)
            sucess = True
            break
        except openai.error.RateLimitError as e:
            print(e)
            time.sleep(10)
            count += 1
        except openai.error.APIConnectionError as e:
            print(e)
            time.sleep(5)
            count += 1
        except openai.error.APIError or openai.error.JSONDecodeError:
            time.sleep(5)
            count += 1
    return response

# ATIS

In [34]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/atis/atis_standard_config.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

In [35]:
dct_phrases = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']

In [11]:
dct_phrases.keys()

dict_keys(['atis_abbreviation', 'atis_aircraft', 'atis_airfare', 'atis_airline', 'atis_flight', 'atis_ground_service'])

In [12]:
dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(default_prompt.format(phrase))

### no taboo words

In [13]:
dct_responses = {}

In [14]:
# collect data

for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(1)

0
Rephrase an original question or statement 3 times. Original phrase: "what is 73s"
Rephrase an original question or statement 3 times. Original phrase: "what does m mean"
Rephrase an original question or statement 3 times. Original phrase: "what does fare code h mean"
Rephrase an original question or statement 3 times. Original phrase: "what is tpa"
Rephrase an original question or statement 3 times. Original phrase: "what does fare code bn mean"
Rephrase an original question or statement 3 times. Original phrase: "what does ap 20 mean"
Rephrase an original question or statement 3 times. Original phrase: "what does q mean"
Rephrase an original question or statement 3 times. Original phrase: "what class is fare code q"
Rephrase an original question or statement 3 times. Original phrase: "what does fare code bh mean"
Rephrase an original question or statement 3 times. Original phrase: "what is cvg"
Rephrase an original question or statement 3 times. Original phrase: "what does ua mean"

Rephrase an original question or statement 3 times. Original phrase: "what are the lowest fares from washington dc to salt lake city"
Rephrase an original question or statement 3 times. Original phrase: "give me the fares for round trip flights from cleveland to miami next wednesday"
Rephrase an original question or statement 3 times. Original phrase: "how much is coach flight from pittsburgh to atlanta"
Rephrase an original question or statement 3 times. Original phrase: "list the cheapest fare from charlotte to las vegas"
Rephrase an original question or statement 3 times. Original phrase: "show me the cheapest round trip coach fare from las vegas to detroit"
Rephrase an original question or statement 3 times. Original phrase: "show me the cheapest one way fare from san francisco to houston on february twenty eighth 1994"
Rephrase an original question or statement 3 times. Original phrase: "cheapest round trip fare from or indianapolis to orlando on december twenty fifth"
Rephrase an

Rephrase an original question or statement 3 times. Original phrase: "which flights leave newark after noon next saturday and arrive in los angeles"
Rephrase an original question or statement 3 times. Original phrase: "st. paul to kansas city friday night"
Rephrase an original question or statement 3 times. Original phrase: "get flights between st. petersburg and charlotte"
Rephrase an original question or statement 3 times. Original phrase: "find flight from cincinnati to san jose on monday"
Rephrase an original question or statement 3 times. Original phrase: "i want a flight from los angeles to charlotte early in the morning"
Rephrase an original question or statement 3 times. Original phrase: "new york to las vegas on sunday afternoon"
Rephrase an original question or statement 3 times. Original phrase: "dallas to houston after 1201 am"
Rephrase an original question or statement 3 times. Original phrase: "show me flights from montreal to orlando"
Rephrase an original question or sta

Rephrase an original question or statement 3 times. Original phrase: "is ground transportation available in st. louis"


In [15]:
with open('challenge_data/atis/atis_0_v1.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [21]:
file = open("challenge_data/atis/atis_0_v1.pkl",'rb')
dct_responses = pickle.load(file)
file.close()

In [22]:
# filter responses to get paraphrases

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])

In [17]:
fb_0 = pd.DataFrame.from_dict(dct_df)

In [18]:
fb_0.to_csv('challenge_data/atis/atis_0.csv', index=False)

### ADD TWO TABOO WORDS

In [13]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/atis/atis_restricted_2_config.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

dct_phrases = {}
dct_taboo = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']
    dct_taboo[key] = data[key]['avoid_words']

# Closing file
f.close()

In [24]:
defaul_taboo_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}". Don’t use the words “{}” or “{}” in your responses.'

In [25]:
# prepare prompts

dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(defaul_taboo_prompt.format(phrase, dct_taboo[key][0], dct_taboo[key][1]))

In [26]:
# collect data

dct_responses = {}
for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(1)

0
Rephrase an original question or statement 3 times. Original phrase: "what is 73s". Don’t use the words “explain” or “mean” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does m mean". Don’t use the words “explain” or “mean” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does fare code h mean". Don’t use the words “explain” or “mean” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is tpa". Don’t use the words “explain” or “mean” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does fare code bn mean". Don’t use the words “explain” or “mean” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does ap 20 mean". Don’t use the words “explain” or “mean” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does q mean". Don’t 

Rephrase an original question or statement 3 times. Original phrase: "get the saturday fare from washington to toronto". Don’t use the words “fares” or “ticket” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "get saturday fares from washington to montreal". Don’t use the words “fares” or “ticket” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "cheapest fare from indianapolis to orlando on the twenty seventh of december". Don’t use the words “fares” or “ticket” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "list first class airfare round trip from indianapolis to memphis". Don’t use the words “fares” or “ticket” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "show me the cheapest round trip fares from houston to boston". Don’t use the words “fares” or “ticket” in your responses.
Rephrase an original question or statement 3 t

Rephrase an original question or statement 3 times. Original phrase: "what airlines fly from baltimore to san francisco". Don’t use the words “airline” or “airlines” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what airline is yx". Don’t use the words “airline” or “airlines” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what airlines fly into denver". Don’t use the words “airline” or “airlines” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "list the airlines with flights to or from denver". Don’t use the words “airline” or “airlines” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "show me airlines that have flights between toronto and detroit between detroit and st. louis and between st. louis and toronto". Don’t use the words “airline” or “airlines” in your responses.
Rephrase an original question or statement 3 times

Rephrase an original question or statement 3 times. Original phrase: "get flights between st. petersburg and charlotte". Don’t use the words “classes” or “flights” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "find flight from cincinnati to san jose on monday". Don’t use the words “classes” or “flights” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "i want a flight from los angeles to charlotte early in the morning". Don’t use the words “classes” or “flights” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "new york to las vegas on sunday afternoon". Don’t use the words “classes” or “flights” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "dallas to houston after 1201 am". Don’t use the words “classes” or “flights” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "show me flights from

Rephrase an original question or statement 3 times. Original phrase: "what ground transportation is available between phoenix airport and downtown phoenix". Don’t use the words “transportation” or “ground” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what ground transportation is available between dca and downtown washington". Don’t use the words “transportation” or “ground” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "is there ground transportation available at the phoenix airport". Don’t use the words “transportation” or “ground” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "is there ground transportation available at the denver airport". Don’t use the words “transportation” or “ground” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what's the ground transportation in westchester county". Don’t use the words “tran

In [27]:
with open('challenge_data/atis/atis_2.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [6]:
file = open("challenge_data/atis/atis_2.pkl",'rb')
dct_responses = pickle.load(file)
file.close()

In [7]:
# filter responses to get paraphrases

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])
                
fb_2 = pd.DataFrame.from_dict(dct_df)
fb_2.to_csv('challenge_data/atis/atis_2.csv', index=False)

In [8]:
fb_2 = pd.DataFrame.from_dict(dct_df)
fb_2 = fb_2.sample(frac=0.5, replace=False, random_state=1)
fb_2.to_csv('challenge_data/atis/atis_2.csv', index=False)

In [14]:
# WE CAN FILTER FOR TABOO WORDS
fb_2 = pd.read_csv('challenge_data/atis/atis_2.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

def are_taboo_words_in(x, y):
    words = x.replace('"', '').split()
    forb_words = dct_taboo[y]
    for word in forb_words:
        if word in words:
            return False
    return True

fb_2['no_taboo_words'] = fb_2.apply(lambda x: are_taboo_words_in(x['text'], x['intent']), axis=1)

fb_2 = fb_2[fb_2['no_taboo_words'] == True]

fb_2.to_csv('challenge_data/atis/atis_2_filt.csv', index=False)

### ADD FOUR TABOO WORDS

In [4]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/atis/atis_restricted_4_config.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

dct_phrases = {}
dct_taboo = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']
    dct_taboo[key] = data[key]['avoid_words']

# Closing file
f.close()

In [37]:
defaul_taboo_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}". Don’t use the words “{}”, “{}”, “{}” or “{}” in your responses.'

In [38]:
# prepare prompts

dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(defaul_taboo_prompt.format(phrase, dct_taboo[key][0], dct_taboo[key][1], dct_taboo[key][2], dct_taboo[key][3]))

In [39]:
#collect data

dct_responses = {}
for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(1)

0
Rephrase an original question or statement 3 times. Original phrase: "what is 73s". Don’t use the words “explain”, “mean”, “stand” or “what” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does m mean". Don’t use the words “explain”, “mean”, “stand” or “what” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does fare code h mean". Don’t use the words “explain”, “mean”, “stand” or “what” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is tpa". Don’t use the words “explain”, “mean”, “stand” or “what” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does fare code bn mean". Don’t use the words “explain”, “mean”, “stand” or “what” in your responses.
That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists

Rephrase an original question or statement 3 times. Original phrase: "get fares from washington to boston". Don’t use the words “fares”, “ticket”, “airfare” or “fare” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "cheapest fare round trip from indianapolis to orlando on december twenty seventh". Don’t use the words “fares”, “ticket”, “airfare” or “fare” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "list coach round trip airfare from detroit to st. petersburg". Don’t use the words “fares”, “ticket”, “airfare” or “fare” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what are the fares for flights between boston and washington dc". Don’t use the words “fares”, “ticket”, “airfare” or “fare” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is the lowest fare from washington dc to montreal". Don’t use the words “fares”, “t

Rephrase an original question or statement 3 times. Original phrase: "what is the lowest fare from bwi to salt lake city". Don’t use the words “fares”, “ticket”, “airfare” or “fare” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "how much does it cost to fly on twa from columbus to milwaukee". Don’t use the words “fares”, “ticket”, “airfare” or “fare” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "get saturday fares from washington to boston". Don’t use the words “fares”, “ticket”, “airfare” or “fare” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "i'd like a one way ticket from milwaukee to orlando either wednesday evening or thursday morning". Don’t use the words “fares”, “ticket”, “airfare” or “fare” in your responses.
3
Rephrase an original question or statement 3 times. Original phrase: "what airlines are departing from baltimore". Don’t use the words “airlin

Rephrase an original question or statement 3 times. Original phrase: "i would like a flight leaving san francisco for san diego". Don’t use the words “classes”, “flights”, “make” or “day” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "flights newark to los angeles wednesday morning". Don’t use the words “classes”, “flights”, “make” or “day” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "i want to fly from san francisco to milwaukee and from milwaukee to denver". Don’t use the words “classes”, “flights”, “make” or “day” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "list the alaska airline flights from burbank". Don’t use the words “classes”, “flights”, “make” or “day” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "get last flight from oakland to salt lake city on wednesday". Don’t use the words “classes”, “flights”, “mak

Rephrase an original question or statement 3 times. Original phrase: "i need a flight from ontario to memphis". Don’t use the words “classes”, “flights”, “make” or “day” in your responses.
5
Rephrase an original question or statement 3 times. Original phrase: "ground transportation for las vegas". Don’t use the words “transportation”, “ground”, “car” or “available” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "i need to rent a car in milwaukee". Don’t use the words “transportation”, “ground”, “car” or “available” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "i need to rent a car at tampa". Don’t use the words “transportation”, “ground”, “car” or “available” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "is there ground transportation from the memphis airport into town when if i arrive at 842 in the morning". Don’t use the words “transportation”, “ground”, “car

In [40]:
with open('challenge_data/atis/atis_4.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [9]:
file = open("challenge_data/atis/atis_4.pkl",'rb')
dct_responses = pickle.load(file)
file.close()

In [41]:
# filter responses to get paraphrases

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])
                
fb_4 = pd.DataFrame.from_dict(dct_df)
fb_4.to_csv('challenge_data/atis/atis_4.csv', index=False)

In [10]:
fb_4 = pd.DataFrame.from_dict(dct_df)
fb_4 = fb_4.sample(frac=0.5, replace=False, random_state=1)
fb_4.to_csv('challenge_data/atis/atis_4.csv', index=False)

In [15]:
# WE CAN FILTER FOR TABOO WORDS
fb_4 = pd.read_csv('challenge_data/atis/atis_4.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

def are_taboo_words_in(x, y):
    words = x.replace('"', '').split()
    forb_words = dct_taboo[y]
    for word in forb_words:
        if word in words:
            return False
    return True

fb_4['no_taboo_words'] = fb_4.apply(lambda x: are_taboo_words_in(x['text'], x['intent']), axis=1)

fb_4 = fb_4[fb_4['no_taboo_words'] == True]

fb_4.to_csv('challenge_data/atis/atis_4_filt.csv', index=False)

### ADD SIX FINAL TABOO WORDS

In [6]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/atis/atis_restricted_6_config.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

dct_phrases = {}
dct_taboo = {}
for key in data:
    dct_phrases[key] = data[key]['phrases']
    dct_taboo[key] = data[key]['avoid_words']

# Closing file
f.close()

In [45]:
defaul_taboo_prompt = 'Rephrase an original question or statement 3 times. Original phrase: "{}". Don’t use the words “{}”, “{}”, “{}”, “{}”, “{}” or “{}” in your responses.'

In [46]:
# prepare prompts

dct_final_prompts = {}

for key in dct_phrases:
    dct_final_prompts[key] = []
    for phrase in dct_phrases[key]:
        dct_final_prompts[key].append(defaul_taboo_prompt.format(phrase, dct_taboo[key][0], dct_taboo[key][1], dct_taboo[key][2], dct_taboo[key][3], dct_taboo[key][4], dct_taboo[key][5]))

In [47]:
# collect data

dct_responses = {}
for idx, key in enumerate(dct_final_prompts):
    print(str(idx))
    dct_responses[key] = []
    for prompt in dct_final_prompts[key]:
        print(prompt)
        response = request_with_checks(prompt)
        dct_responses[key].append(response)
        time.sleep(1)

0
Rephrase an original question or statement 3 times. Original phrase: "what is 73s". Don’t use the words “explain”, “mean”, “stand”, “what”, “code” or “is” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does m mean". Don’t use the words “explain”, “mean”, “stand”, “what”, “code” or “is” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does fare code h mean". Don’t use the words “explain”, “mean”, “stand”, “what”, “code” or “is” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is tpa". Don’t use the words “explain”, “mean”, “stand”, “what”, “code” or “is” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does fare code bn mean". Don’t use the words “explain”, “mean”, “stand”, “what”, “code” or “is” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what does ap 20 mean".

Rephrase an original question or statement 3 times. Original phrase: "get fares from washington to boston". Don’t use the words “fares”, “ticket”, “airfare”, “fare”, “prices” or “tickets” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "cheapest fare round trip from indianapolis to orlando on december twenty seventh". Don’t use the words “fares”, “ticket”, “airfare”, “fare”, “prices” or “tickets” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "list coach round trip airfare from detroit to st. petersburg". Don’t use the words “fares”, “ticket”, “airfare”, “fare”, “prices” or “tickets” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what are the fares for flights between boston and washington dc". Don’t use the words “fares”, “ticket”, “airfare”, “fare”, “prices” or “tickets” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what

Rephrase an original question or statement 3 times. Original phrase: "cheapest one way fare from indianapolis to orlando on december twenty seventh". Don’t use the words “fares”, “ticket”, “airfare”, “fare”, “prices” or “tickets” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "list fares from washington dc to montreal". Don’t use the words “fares”, “ticket”, “airfare”, “fare”, “prices” or “tickets” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what is the lowest fare from bwi to salt lake city". Don’t use the words “fares”, “ticket”, “airfare”, “fare”, “prices” or “tickets” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "how much does it cost to fly on twa from columbus to milwaukee". Don’t use the words “fares”, “ticket”, “airfare”, “fare”, “prices” or “tickets” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "get saturday

Rephrase an original question or statement 3 times. Original phrase: "show flights from burbank to st. louis on monday". Don’t use the words “classes”, “flights”, “make”, “day”, “flight” or “latest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "list a flight from toronto to san diego". Don’t use the words “classes”, “flights”, “make”, “day”, “flight” or “latest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "please list all the flights from dallas to san francisco". Don’t use the words “classes”, “flights”, “make”, “day”, “flight” or “latest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "list flights from atlanta to toronto friday afternoon". Don’t use the words “classes”, “flights”, “make”, “day”, “flight” or “latest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what flights leave from cincinnati in the morning and 

Rephrase an original question or statement 3 times. Original phrase: "list flights from houston to milwaukee". Don’t use the words “classes”, “flights”, “make”, “day”, “flight” or “latest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "next wednesday i would like to leave kansas city on a trip to chicago which arrives in chicago around 7 pm". Don’t use the words “classes”, “flights”, “make”, “day”, “flight” or “latest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "first class flights pittsburgh to newark monday morning". Don’t use the words “classes”, “flights”, “make”, “day”, “flight” or “latest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "please list the flights from newark to los angeles". Don’t use the words “classes”, “flights”, “make”, “day”, “flight” or “latest” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "s

Rephrase an original question or statement 3 times. Original phrase: "show me ground transportation in fort worth". Don’t use the words “transportation”, “ground”, “car”, “available”, “transport” or “airport” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "what's the ground transportation in denver". Don’t use the words “transportation”, “ground”, “car”, “available”, “transport” or “airport” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "and now show me ground transportation that i could get in boston late night". Don’t use the words “transportation”, “ground”, “car”, “available”, “transport” or “airport” in your responses.
Rephrase an original question or statement 3 times. Original phrase: "tell me about the ground transportation from nashville airport". Don’t use the words “transportation”, “ground”, “car”, “available”, “transport” or “airport” in your responses.
Rephrase an original question or state

In [48]:
with open('challenge_data/atis/atis_6.pkl', 'wb') as file:
    pickle.dump(dct_responses, file)

In [11]:
file = open("challenge_data/atis/atis_6.pkl",'rb')
dct_responses = pickle.load(file)
file.close()

In [49]:
# filter responses to get paraphrases

dct_df = {'intent': [], 'text': []}
for key in dct_responses:
    for responses in dct_responses[key]:
        for response in responses['choices']:
            contents = response['message']['content'].split('\n')
            for content in contents:
                dct_df['intent'].append(key)
                dct_df['text'].append(content[3:])
                
fb_6 = pd.DataFrame.from_dict(dct_df)
fb_6.to_csv('challenge_data/atis/atis_6.csv', index=False)

In [12]:
fb_6 = pd.DataFrame.from_dict(dct_df)
fb_6 = fb_6.sample(frac=0.5, replace=False, random_state=1)
fb_6.to_csv('challenge_data/atis/atis_6.csv', index=False)

In [16]:
# WE CAN FILTER FOR TABOO WORDS
fb_6 = pd.read_csv('challenge_data/atis/atis_6.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

def are_taboo_words_in(x, y):
    words = x.replace('"', '').split()
    forb_words = dct_taboo[y]
    for word in forb_words:
        if word in words:
            return False
    return True

fb_6['no_taboo_words'] = fb_6.apply(lambda x: are_taboo_words_in(x['text'], x['intent']), axis=1)

fb_6 = fb_6[fb_6['no_taboo_words'] == True]

fb_6.to_csv('challenge_data/atis/atis_6_filt.csv', index=False)

## PREPARE DATASET 

In [17]:
# filtered data without taboo samples

fb_0 = pd.read_csv('challenge_data/atis/atis_0.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_2 = pd.read_csv('challenge_data/atis/atis_2_filt.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_4 = pd.read_csv('challenge_data/atis/atis_4_filt.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_6 = pd.read_csv('challenge_data/atis/atis_6_filt.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

In [ ]:
# data with taboo samples

fb_0 = pd.read_csv('challenge_data/atis/atis_0.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_2 = pd.read_csv('challenge_data/atis/atis_2.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_4 = pd.read_csv('challenge_data/atis/atis_4.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)
fb_6 = pd.read_csv('challenge_data/atis/atis_6.csv').dropna().reset_index(drop=True).sample(frac=1, replace=False, random_state=1)

Proceed with either taboo samples or without them

In [19]:
dct = {'atis_abbreviation': 0,
 'atis_aircraft': 1,
 'atis_airfare': 2,
 'atis_airline': 3,
 'atis_flight': 4,
 'atis_ground_service': 5}

def function(x):
    return dct[x]

fb['label'] = fb['intent'].apply(function)
fb = fb.sample(frac=1).reset_index(drop=True)
fb['text'] = fb['text'].apply(lambda x: x.replace('"',''))

In [20]:
fb[['text', 'label']].to_csv('challenge_data/atis/full_atis_gpt.csv', index=False)

Now lets prepare user data and the original data into dataset - for human data we have the same preprocessing and for the  original we remove the seed samples, take only needed intents and prepare csv

To get the data for the original version and human version from the original paper, please see the README.md file at the root

In [19]:
import json 

# Opening JSON file
dct_dataframe = {'text': [], 'intent': []}

f = open('/taboo_data/challenge_data/intent_classification/atis/atis_standard_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])
    
fb_h_0 = pd.DataFrame.from_dict(dct_dataframe)

In [20]:
f = open('/taboo_data/challenge_data/intent_classification/atis/atis_restricted_2_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])

fb_h_2 = pd.DataFrame.from_dict(dct_dataframe)

f = open('/taboo_data/challenge_data/intent_classification/atis/atis_restricted_4_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])

fb_h_4 = pd.DataFrame.from_dict(dct_dataframe)


f = open('/taboo_data/challenge_data/intent_classification/atis/atis_restricted_6_test.json')

# returns JSON object as 
# a dictionary
data = json.load(f)

# Closing file
f.close()

dct_dataframe = {'text': [], 'intent': []}

for item in data['test']:
    dct_dataframe['intent'].append(item[1])
    dct_dataframe['text'].append(item[0])

fb_h_6 = pd.DataFrame.from_dict(dct_dataframe)

In [25]:
fb_h = fb_h_0.append(fb_h_2).append(fb_h_4).append(fb_h_6).drop_duplicates('text')

In [27]:
fb_h['label'] = fb_h['intent'].apply(function)
fb_h = fb_h.sample(frac=1).reset_index(drop=True)
fb_h['text'] = fb_h['text'].apply(lambda x: x.replace('"',''))

In [28]:
fb_h[['text', 'label']].to_csv('full_atis_human.csv', index=False)

Now original data without the seed data

In [30]:
intents = list(set(fb_h['intent']))

In [55]:

f = open('/taboo_data/challenge_data/intent_classification/atis/train.json')

# returns JSON object as 
# a dictionary
data = json.load(f)


# Closing file
f.close()

In [45]:
all_seed_phrases = []
for key in dct_phrases:
    all_seed_phrases.extend(dct_phrases[key])

In [56]:
fin_dct = {}
for key in data.keys():
    if key not in all_seed_phrases:
        fin_dct[key] = data[key]

In [59]:
dct_dataframe = {'text': [], 'intent': []}

for key in data:
    dct_dataframe['intent'].append(data[key])
    dct_dataframe['text'].append(key)

tmp_orig = pd.DataFrame.from_dict(dct_dataframe)
tmp_orig = tmp_orig[tmp_orig['intent'].isin(intents)]
tmp_orig = tmp_orig.sample(frac=1).reset_index(drop=True)
tmp_orig['label'] = tmp_orig['intent'].apply(function)

tmp_orig[['text', 'label']].to_csv('full_atis_orig_train.csv', index=False)

## LETS DO SVM Training 

In [61]:
full_gpt = pd.read_csv('challenge_data/atis/full_atis_gpt.csv').dropna().reset_index(drop=True)
full_human = pd.read_csv('challenge_data/atis/full_atis_human.csv').dropna().reset_index(drop=True)
full_orig = pd.read_csv('challenge_data/atis/full_atis_orig_train.csv').dropna().reset_index(drop=True)

full_orig_test = pd.read_csv('challenge_data/atis/full_atis_orig_test.csv').dropna().reset_index(drop=True)

In [62]:
import re
import string

full_gpt['text']=full_gpt['text'].apply(lambda x: x.lower())
full_human['text']=full_human['text'].apply(lambda x: x.lower())
full_orig['text']=full_orig['text'].apply(lambda x: x.lower())
full_orig_test['text']=full_orig_test['text'].apply(lambda x: x.lower())

full_gpt['text']=full_gpt['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
full_human['text']=full_human['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
full_orig['text']=full_orig['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))
full_orig_test['text']=full_orig_test['text'].apply(lambda x: re.sub('[%s]' % re.escape(string.punctuation), '', x))

full_gpt['text']=full_gpt['text'].apply(lambda x: re.sub(' +',' ',x))
full_human['text']=full_human['text'].apply(lambda x: re.sub(' +',' ',x))
full_orig['text']=full_orig['text'].apply(lambda x: re.sub(' +',' ',x))
full_orig_test['text']=full_orig_test['text'].apply(lambda x: re.sub(' +',' ',x))

In [67]:
import pandas as pd
import nltk
import re
from nltk.corpus import stopwords
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report,confusion_matrix
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.neural_network import MLPClassifier
from joblib import dump, load
from sklearn import tree
import numpy as np
import itertools
from sklearn import svm
from sklearn.model_selection import train_test_split

gpt_res = []
gpt_f1 = []

human_res = []
human_f1 = []

orig_res = []
orig_f1 = []

for idx in range(0, 10):

    train_gpt, test_gpt = train_test_split(full_gpt, test_size=0.2)
    train_human, test_human = train_test_split(full_human, test_size=0.2)
    
    tmp_train = full_orig.sample(frac=0.2, replace=False, random_state=1)
    tmp_test = full_orig_test.sample(frac=1, replace=False, random_state=1)
    
    pipeline = Pipeline([
            ('bow', CountVectorizer()),  
            ('tfidf', TfidfTransformer()),  
            ('c', svm.SVC(probability=True))
        ])
    fit = pipeline.fit(full_gpt['text'].str.lower(), full_gpt['label'])
    
    pred=pipeline.predict(test_gpt['text'].str.lower())
    gpt_res.append(accuracy_score(pred,test_gpt['label']))
    gpt_f1.append(f1_score(pred,test_gpt['label'],  average = 'weighted'))
    
    pred=pipeline.predict(test_human['text'].str.lower())
    human_res.append(accuracy_score(pred,test_human['label']))
    human_f1.append(f1_score(pred,test_human['label'],  average = 'weighted'))
    
    pred=pipeline.predict(tmp_test['text'].str.lower())
    orig_res.append(accuracy_score(pred,tmp_test['label']))
    orig_f1.append(f1_score(pred,tmp_test['label'],  average = 'weighted'))

In [68]:
arr = np.array(gpt_res)

print("ACC GPT MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(gpt_f1)

print("F1 GPT MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr))) 

arr = np.array(human_res)

print("ACC HUMAN SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(human_f1)

print("F1 HUMAN SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(orig_res)

print("ACC ORIG SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

arr = np.array(orig_f1)

print("F1 ORIG SAME MEAN: " + str(np.mean(arr)) + " STD: " + str(np.std(arr)))

ACC GPT MEAN: 0.9975295381310418 STD: 0.001275439536738768
F1 GPT MEAN: 0.9975317990543715 STD: 0.001273120410171808
ACC HUMAN SAME MEAN: 0.9624728850325381 STD: 0.00878725774426426
F1 HUMAN SAME MEAN: 0.962570557189618 STD: 0.00878446903345318
ACC ORIG SAME MEAN: 0.8142292490118577 STD: 0.0
F1 ORIG SAME MEAN: 0.7834866980280959 STD: 0.0
